In [1]:
import pandas as pd
import numpy as np
import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [5]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/new_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels20.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno

In [49]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
            tuned_dlm, parameters = pt.KFoldGridSearch(
                dlm,
                {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_dlm, dlm], topics, qrels, 
                                names=['tuned_dlm','orignal_dlm'], save_mode = "overwrite", 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"],  save_dir = "/workspace/src/runs_dlm"))
            
            print(pt.Experiment([tuned_dlm, dlm], topics20, qrels, names=['20original_tuned_dlm', '20orignal_dlm'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            
            print(pt.Experiment([tuned_dlm, dlm], topics21, qrels, names=['21original_tuned_dlm', '21orignal_dlm'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            tuned_dlm.to_csv(path_or_buf=f"/workspace/src/runs_dlm/original_tuned_dlm.csv")
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.652732
Best setting is ['BR(DirichletLM) dirichletlm.mu=3000']
Fold 2
Best ndcg_cut_5 is 0.467390
Best setting is ['BR(DirichletLM) dirichletlm.mu=4250']
          name  ndcg_cut_5  ndcg_cut_25     bpref
0    tuned_dlm    0.555781     0.391685  0.508813
1  orignal_dlm    0.552341     0.401122  0.513215
                   name  ndcg_cut_5  ndcg_cut_25     bpref
0  20original_tuned_dlm    0.461558     0.282011  0.421595
1         20orignal_dlm    0.463348     0.282512  0.425434
                   name  ndcg_cut_5  ndcg_cut_25     bpref
0  21original_tuned_dlm    0.648120     0.499166  0.594287
1         21orignal_dlm    0.639554     0.517360  0.599241


In [53]:
'''
Example of simple tuning dlm and its not saving to the folder 
'''

import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
metrics="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dlm = pt.BatchRetrieve(index, wmodel="DirichletLM", controls={"dirichletlm.mu" : 2500})
            tuned_dlm, parameters = pt.KFoldGridSearch(
                dlm,
                {dlm: {"dirichletlm.mu" : list(range(0,10000,250))}},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_dlm, dlm], topics, qrels, names=[folders[i] + '_tuned_dlm', folders[i] + '_orignal_dlm'], save_mode = "overwrite", eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"],  save_dir = "/workspace/src/runs_dlm"))
            
            print(pt.Experiment([tuned_dlm, dlm], topics20, qrels, names=['20' + folders[i] + 'tuned_dlm', '20' + folders[i] + '_dlm'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            
            print(pt.Experiment([tuned_dlm, dlm], topics21, qrels, names=['21' + folders[i] + 'tuned_dlm', '21'+ folders[i] + '_dlm'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            
            tuned_dlm.to_csv(path_or_buf=f"/workspace/src/runs_dlm/{folders[i]}_tuned_dlm.csv")
            i += 1
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.607973
Best setting is ['BR(DirichletLM) dirichletlm.mu=3750']
Fold 2
Best ndcg_cut_5 is 0.391740
Best setting is ['BR(DirichletLM) dirichletlm.mu=5750']
                      name  ndcg_cut_5  ndcg_cut_25     bpref
0    conclusions_tuned_dlm    0.492296     0.389321  0.592431
1  conclusions_orignal_dlm    0.489808     0.394094  0.599061
                     name  ndcg_cut_5  ndcg_cut_25     bpref
0  20conclusionstuned_dlm    0.382045     0.262577  0.462891
1       20conclusions_dlm    0.372117     0.261202  0.469089
                     name  ndcg_cut_5  ndcg_cut_25     bpref
0  21conclusionstuned_dlm    0.600342     0.513530  0.719380
1       21conclusions_dlm    0.605145     0.524329  0.726433

/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/pool_topicspd_index
Fold 1
Best ndcg_cut_5 is 0.624379
Best 